    000_16_230114_141_subway_transer_22
    
    data : 022_분당선_주소데이터_20221122.csv
     save : subway_info_22.csv
     display : subway_info_22.html

In [46]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [47]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [48]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [49]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/022_분당선_주소데이터_20221122.csv',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,코레일,수인분당,가천대,경기도 성남시 수정구 태평동 7131,경기도 성남시 수정구 성남대로 1332,NaN,NaN,NaN,NaN,NaN,NaN
1,코레일,수인분당,강남구청,서울 강남구 삼성동 77-94,서울특별시 강남구 학동로 346,NaN,NaN,NaN,NaN,NaN,NaN
2,코레일,수인분당,개포동,서울시 강남구 개포동 181-2,서울시 강남구 개포로 420,NaN,NaN,NaN,NaN,NaN,NaN
3,코레일,수인분당,고색,경기도 수원시 권선구 고색동 377-2,경기도 수원시 권선구 매송고색로 690,NaN,NaN,NaN,NaN,NaN,NaN
4,코레일,수인분당,고잔,경기도 안산시 단원구 고잔동 453-67,경기도 안산시 단원구 중앙대로 784,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   철도운영기관명      63 non-null     object 
 1   선명           63 non-null     object 
 2   역명           63 non-null     object 
 3   지번주소         63 non-null     object 
 4   도로명주소        63 non-null     object 
 5   Unnamed: 5   0 non-null      float64
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
dtypes: float64(6), object(5)
memory usage: 5.5+ KB


In [51]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [52]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [53]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["도로명주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["도로명주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,x,y
0,코레일,수인분당,가천대,경기도 성남시 수정구 태평동 7131,경기도 성남시 수정구 성남대로 1332,NaN,NaN,NaN,NaN,NaN,NaN,966981.769194114,1938886.55691386
1,코레일,수인분당,강남구청,서울 강남구 삼성동 77-94,서울특별시 강남구 학동로 346,NaN,NaN,NaN,NaN,NaN,NaN,959463.746268146,1946532.42905012
2,코레일,수인분당,개포동,서울시 강남구 개포동 181-2,서울시 강남구 개포로 420,NaN,NaN,NaN,NaN,NaN,NaN,961664.538135734,1943421.80544968
3,코레일,수인분당,고색,경기도 수원시 권선구 고색동 377-2,경기도 수원시 권선구 매송고색로 690,NaN,NaN,NaN,NaN,NaN,NaN,953927.361502964,1916888.21747733
4,코레일,수인분당,고잔,경기도 안산시 단원구 고잔동 453-67,경기도 안산시 단원구 중앙대로 784,NaN,NaN,NaN,NaN,NaN,NaN,940026.184624752,1924417.61808282


In [54]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [55]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,x,y,lng,lat
0,코레일,수인분당,가천대,경기도 성남시 수정구 태평동 7131,경기도 성남시 수정구 성남대로 1332,NaN,NaN,NaN,NaN,NaN,NaN,966981.769194114,1938886.55691386,127.126705,37.448575
1,코레일,수인분당,강남구청,서울 강남구 삼성동 77-94,서울특별시 강남구 학동로 346,NaN,NaN,NaN,NaN,NaN,NaN,959463.746268146,1946532.42905012,127.041290,37.517192
2,코레일,수인분당,개포동,서울시 강남구 개포동 181-2,서울시 강남구 개포로 420,NaN,NaN,NaN,NaN,NaN,NaN,961664.538135734,1943421.80544968,127.066355,37.489248
3,코레일,수인분당,고색,경기도 수원시 권선구 고색동 377-2,경기도 수원시 권선구 매송고색로 690,NaN,NaN,NaN,NaN,NaN,NaN,953927.361502964,1916888.21747733,126.980489,37.249734
4,코레일,수인분당,고잔,경기도 안산시 단원구 고잔동 453-67,경기도 안산시 단원구 중앙대로 784,NaN,NaN,NaN,NaN,NaN,NaN,940026.184624752,1924417.61808282,126.823141,37.316808


In [56]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [57]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [58]:
#5 seoul university 
map

In [59]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_22.html")


In [60]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_22.csv", index = False)
